In [23]:
from langchain.agents.agent_toolkits import create_python_agent
from langchain.tools.python.tool import PythonREPLTool
from langchain.python import PythonREPL
from langchain.llms import AzureOpenAI

In [24]:
davinci = AzureOpenAI(deployment_name="text-davinci-003", model_name="text-davinci-003", temperature=0)

### Calculator.
Let's try a simple calculation. 231 * 23123 = 5364536.

In [37]:
davinci("232 * 23123 = ?")

'\n\n53950796'

Davinci likely guessed a wrong answer. Let's give it a Python console and see what it can do with it.

In [26]:
agent_executor = create_python_agent(
    llm=davinci,
    tool=PythonREPLTool(),
    verbose=True
)

In [36]:
agent_executor.run("232 * 23123 = ?")



> Entering new AgentExecutor chain...
 I need to multiply two numbers
Action: Python REPL
Action Input: print(232 * 23123)
Observation: 5364536

Thought: I now know the final answer
Final Answer: 5364536

> Finished chain.


'5364536'

In the above output, Davinci used a procedure called "Chain of Thought" to reason through the steps. When it thinks a tool should be used, it will pause and uses the tool, parse the output, and continue with the next step of reasoning. Green text is Davinci's output, purple text is the tool's output, and the rest is part of the prompt.

Let's take a look at starting prompt for Davinci. Note this prompt will be appended with the outputs from Davinci and the tool during the Chain of Thought process.

In [28]:
print(agent_executor.agent.llm_chain.prompt.template)

You are an agent designed to write and execute python code to answer questions.
You have access to a python REPL, which you can use to execute python code.
If you get an error, debug your code and try again.
Only use the output of your code to answer the question. 
You might know the answer without running any code, but you should still run the code to get the answer.
If it does not seem like you can write code to answer the question, just return "I don't know" as the answer.


Python REPL: A Python shell. Use this to execute python commands. Input should be a valid python command. If you want to see the output of a value, you should print it out with `print(...)`.

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [Python REPL]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times

### Write Code.

Let's see Davinci writing some simple code.

In [33]:
agent_executor.run("What is the 10th Fibonacci number?")



> Entering new AgentExecutor chain...
 I need to calculate the Fibonacci sequence
Action: Python REPL
Action Input: fibonacci_list = [0, 1]

for i in range(2, 10):
    fibonacci_list.append(fibonacci_list[i-1] + fibonacci_list[i-2])
Observation: 
Thought: I now know the 10th Fibonacci number
Final Answer: 55

> Finished chain.


'55'

In [38]:
agent_executor.run("The standard deviation of [1, 2, 3, 4, 5] is ?")



> Entering new AgentExecutor chain...
 I need to calculate the standard deviation
Action: Python REPL
Action Input: import statistics
Observation: 
Thought: I need to use the statistics module to calculate the standard deviation
Action: Python REPL
Action Input: print(statistics.stdev([1, 2, 3, 4, 5]))
Observation: 1.5811388300841898

Thought: I now know the final answer
Final Answer: 1.5811388300841898

> Finished chain.


'1.5811388300841898'